# Import Required Packages

In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [117]:
#!pip install torchvision
#!pip install opencv-python

# Download Files

In [118]:
#training controls
batch_size = 64
epochs = 15
training_size = 0.7
learning_rate = 0.0001

# input image dimensions
img_rows, img_cols = 268, 182

In [119]:
# data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [120]:
# opening the dataset
dataset = csv.reader(open("MovieGenre3.csv",encoding="utf8",errors='replace'), delimiter=",")

# skipping the header line
next(dataset)

['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [121]:
# extract images from zip folder

import zipfile as zf

files = zf.ZipFile("Poster.zip", 'r')
files.extractall()
files.close()

In [122]:
# list of image files in SampleMoviePosters folder
flist=glob.glob('Poster/*.jpg')  

In [123]:
len(flist)

1354

In [124]:
image_ids = []

for path in flist:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [125]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre3.csv")
dataset2

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,160905,http://www.imdb.com/title/tt160905,Spooky House (2002),5.4,Comedy|Family,https://images-na.ssl-images-amazon.com/images...
1,427531,http://www.imdb.com/title/tt427531,Mezzo Forte,6.9,Animation|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...
2,4175088,http://www.imdb.com/title/tt4175088,Radical Grace (2015),8.3,Documentary|News,https://images-na.ssl-images-amazon.com/images...
3,1686053,http://www.imdb.com/title/tt1686053,Apnoia (2010),5.6,Drama|Thriller,https://images-na.ssl-images-amazon.com/images...
4,22921,http://www.imdb.com/title/tt22921,Broadway to Cheyenne (1932),4.8,Western,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
1349,821638,http://www.imdb.com/title/tt821638,Bury My Heart at Wounded Knee (2007),7.2,Drama|History|Western,https://images-na.ssl-images-amazon.com/images...
1350,87835,http://www.imdb.com/title/tt87835,"Oh, God! You Devil (1984)",5.3,Comedy|Fantasy,https://images-na.ssl-images-amazon.com/images...
1351,103710,http://www.imdb.com/title/tt103710,AprÌ¬s l'amour (1992),6.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...
1352,108265,http://www.imdb.com/title/tt108265,Swing Kids (1993),6.8,Drama|Music,https://images-na.ssl-images-amazon.com/images...


# Data Preprocessing

In [126]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [127]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Musical,Mystery,News,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
87913,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2788556,0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4767340,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
363473,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
393775,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018920,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
244244,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1897945,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
377569,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,160905,http://www.imdb.com/title/tt160905,Spooky House (2002),5.4,Comedy|Family,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,427531,http://www.imdb.com/title/tt427531,Mezzo Forte,6.9,Animation|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4175088,http://www.imdb.com/title/tt4175088,Radical Grace (2015),8.3,Documentary|News,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1686053,http://www.imdb.com/title/tt1686053,Apnoia (2010),5.6,Drama|Thriller,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,22921,http://www.imdb.com/title/tt22921,Broadway to Cheyenne (1932),4.8,Western,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1349,821638,http://www.imdb.com/title/tt821638,Bury My Heart at Wounded Knee (2007),7.2,Drama|History|Western,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ..."
1350,87835,http://www.imdb.com/title/tt87835,"Oh, God! You Devil (1984)",5.3,Comedy|Fantasy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1351,103710,http://www.imdb.com/title/tt103710,AprÌ¬s l'amour (1992),6.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1352,108265,http://www.imdb.com/title/tt108265,Swing Kids (1993),6.8,Drama|Music,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."


In [129]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

# Train/Test Split

In [130]:
#setting the length of training data
length=int(len(flist)*training_size)
length

947

In [131]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('/')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [132]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [133]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [134]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (947, 3, 268, 182)
947 train samples
407 test samples


In [135]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [136]:
# Metric calculation

def metric(scores, targets):
    """
    :param scores: the output the model predict
    :param targets: the gt label
    :return: OP, OR, OF1, CP, CR, CF1
    calculate the Precision of every class by: TP/TP+FP i.e. TP/total predict
    calculate the Recall by: TP/total GT
    """
    num, num_class = scores.shape
    gt_num = np.zeros(num_class)
    tp_num = np.zeros(num_class)
    predict_num = np.zeros(num_class)


    for index in range(num_class):
        score = scores[:, index]
        target = targets[:, index]

        gt_num[index] = np.sum(target == 1)
        predict_num[index] = np.sum(score >= 0.5)
        tp_num[index] = np.sum(target * (score >= 0.5))

    predict_num[predict_num == 0] = 1  # avoid dividing 0
    OP = np.sum(tp_num) / np.sum(predict_num) #OP (Overall Precision) is the ratio of the number of correctly predicted positive samples to the total number of positive predictions made by the model
    OR = np.sum(tp_num) / np.sum(gt_num) #OR (Overall Recall) is the ratio of the number of correctly predicted positive samples to the total number of positive samples in the ground truth.
    OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.

    return OP, OR, OF1

# Model 1: Resnet50

In [72]:
# Resnet50 model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))

    def forward(self, x):
        x = self.resnet(x)
        return x

In [73]:
model = ResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [74]:
from statistics import mean
import numpy as np

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/947 (0%)]	Loss: 0.694314 	OP: 0.020408	OR: 0.015385	OF1: 0.017544


<ipython-input-71-447663f93c68>:28: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [32/947 (3%)]	Loss: 0.405906 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [64/947 (7%)]	Loss: 0.302793 	OP: 0.041667	OR: 0.014706	OF1: 0.021739
Train Epoch: 0 [96/947 (10%)]	Loss: 0.295557 	OP: 0.125000	OR: 0.052632	OF1: 0.074074
Train Epoch: 0 [128/947 (13%)]	Loss: 0.272327 	OP: 0.068966	OR: 0.029412	OF1: 0.041237
Train Epoch: 0 [160/947 (17%)]	Loss: 0.276444 	OP: 0.205882	OR: 0.100000	OF1: 0.134615
Train Epoch: 0 [192/947 (20%)]	Loss: 0.256322 	OP: 0.272727	OR: 0.140625	OF1: 0.185567
Train Epoch: 0 [224/947 (23%)]	Loss: 0.312750 	OP: 0.129032	OR: 0.055556	OF1: 0.077670
Train Epoch: 0 [256/947 (27%)]	Loss: 0.292756 	OP: 0.142857	OR: 0.070423	OF1: 0.094340
Train Epoch: 0 [288/947 (30%)]	Loss: 0.237431 	OP: 0.083333	OR: 0.032258	OF1: 0.046512
Train Epoch: 0 [320/947 (33%)]	Loss: 0.293724 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [352/947 (37%)]	Loss: 0.264802 	OP: 0.040000	OR: 0.013699	OF1: 0.020408
Train Epoch: 0 [384/947 (40%)]	Loss: 0.254141 	OP: 0.04166

<ipython-input-74-ea5e5cd9fa3b>:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 1.3307 
OP: 0.194444
OR: 0.132075
OF1: 0.157303

Train Epoch: 1 [0/947 (0%)]	Loss: 0.245240 	OP: 0.074074	OR: 0.031746	OF1: 0.044444
Train Epoch: 1 [32/947 (3%)]	Loss: 0.234321 	OP: 0.161290	OR: 0.070423	OF1: 0.098039
Train Epoch: 1 [64/947 (7%)]	Loss: 0.253127 	OP: 0.187500	OR: 0.084507	OF1: 0.116505
Train Epoch: 1 [96/947 (10%)]	Loss: 0.233698 	OP: 0.107143	OR: 0.044776	OF1: 0.063158
Train Epoch: 1 [128/947 (13%)]	Loss: 0.247340 	OP: 0.178571	OR: 0.070423	OF1: 0.101010
Train Epoch: 1 [160/947 (17%)]	Loss: 0.254522 	OP: 0.111111	OR: 0.043478	OF1: 0.062500
Train Epoch: 1 [192/947 (20%)]	Loss: 0.245844 	OP: 0.115385	OR: 0.042857	OF1: 0.062500
Train Epoch: 1 [224/947 (23%)]	Loss: 0.257143 	OP: 0.142857	OR: 0.051948	OF1: 0.076190
Train Epoch: 1 [256/947 (27%)]	Loss: 0.242512 	OP: 0.142857	OR: 0.059701	OF1: 0.084211
Train Epoch: 1 [288/947 (30%)]	Loss: 0.237800 	OP: 0.161290	OR: 0.072464	OF1: 0.100000
Train Epoch: 1 [320/947 (33%)]	Loss: 0.234496 	OP: 0.218750	

KeyboardInterrupt: 

# Model 2: DenseNet201

In [53]:
# DenseNet201

import torchvision
import torch.nn as nn

model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model2.classifier.in_features
model2.classifier = nn.Linear(num_ftrs, len(classes))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [54]:
from statistics import mean
import numpy as np

def train(epoch):
    model2.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model2(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'wider')
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model2.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model2(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'wider')

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/1083 (0%)]	Loss: 0.732575 	OP: 0.088608	OR: 0.097222	OF1: 0.092715
Train Epoch: 0 [32/1083 (3%)]	Loss: 0.707817 	OP: 0.136986	OR: 0.133333	OF1: 0.135135
Train Epoch: 0 [64/1083 (6%)]	Loss: 0.680108 	OP: 0.140000	OR: 0.101449	OF1: 0.117647
Train Epoch: 0 [96/1083 (9%)]	Loss: 0.667057 	OP: 0.069767	OR: 0.048387	OF1: 0.057143
Train Epoch: 0 [128/1083 (12%)]	Loss: 0.645402 	OP: 0.027778	OR: 0.014493	OF1: 0.019048
Train Epoch: 0 [160/1083 (15%)]	Loss: 0.611889 	OP: 0.138889	OR: 0.079365	OF1: 0.101010
Train Epoch: 0 [192/1083 (18%)]	Loss: 0.600936 	OP: 0.103448	OR: 0.042857	OF1: 0.060606
Train Epoch: 0 [224/1083 (21%)]	Loss: 0.581095 	OP: 0.040000	OR: 0.015385	OF1: 0.022222
Train Epoch: 0 [256/1083 (24%)]	Loss: 0.567764 	OP: 0.038462	OR: 0.014085	OF1: 0.020619
Train Epoch: 0 [288/1083 (26%)]	Loss: 0.546047 	OP: 0.040000	OR: 0.015152	OF1: 0.021978
Train Epoch: 0 [320/1083 (29%)]	Loss: 0.524420 	OP: 0.148148	OR: 0.055556	OF1: 0.080808
Train Epoch: 0 [352/1083 (32%)]	Loss: 0.5

<ipython-input-52-bbf89c8d258d>:40: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [864/1083 (79%)]	Loss: 0.342054 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [896/1083 (82%)]	Loss: 0.334149 	OP: 0.153846	OR: 0.054795	OF1: 0.080808
Train Epoch: 0 [928/1083 (85%)]	Loss: 0.329278 	OP: 0.080000	OR: 0.029851	OF1: 0.043478
Train Epoch: 0 [960/1083 (88%)]	Loss: 0.321470 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [992/1083 (91%)]	Loss: 0.305416 	OP: 0.041667	OR: 0.014706	OF1: 0.021739
Train Epoch: 0 [1024/1083 (94%)]	Loss: 0.314984 	OP: 0.080000	OR: 0.027778	OF1: 0.041237
Train Epoch: 0 [891/1083 (97%)]	Loss: 0.315591 	OP: 0.000000	OR: 0.000000	OF1: nan
test


<ipython-input-54-78eea7326411>:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.3080 
OP: 0.040000
OR: 0.028571
OF1: 0.033333

Train Epoch: 1 [0/1083 (0%)]	Loss: 0.273257 	OP: 0.153846	OR: 0.058824	OF1: 0.085106
Train Epoch: 1 [32/1083 (3%)]	Loss: 0.261681 	OP: 0.206897	OR: 0.090909	OF1: 0.126316
Train Epoch: 1 [64/1083 (6%)]	Loss: 0.244387 	OP: 0.333333	OR: 0.180328	OF1: 0.234043
Train Epoch: 1 [96/1083 (9%)]	Loss: 0.249878 	OP: 0.290323	OR: 0.145161	OF1: 0.193548
Train Epoch: 1 [128/1083 (12%)]	Loss: 0.238852 	OP: 0.241379	OR: 0.106061	OF1: 0.147368
Train Epoch: 1 [160/1083 (15%)]	Loss: 0.241438 	OP: 0.153846	OR: 0.058824	OF1: 0.085106
Train Epoch: 1 [192/1083 (18%)]	Loss: 0.250312 	OP: 0.206897	OR: 0.082192	OF1: 0.117647
Train Epoch: 1 [224/1083 (21%)]	Loss: 0.224105 	OP: 0.371429	OR: 0.213115	OF1: 0.270833
Train Epoch: 1 [256/1083 (24%)]	Loss: 0.219151 	OP: 0.266667	OR: 0.125000	OF1: 0.170213
Train Epoch: 1 [288/1083 (26%)]	Loss: 0.237909 	OP: 0.153846	OR: 0.058824	OF1: 0.085106
Train Epoch: 1 [320/1083 (29%)]	Loss: 0.234306 	OP:

# Model 3: ResNet18

In [80]:
# Resnet152 model
from torchvision import models

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))

    def forward(self, x):
        x = self.resnet(x)
        return x

In [81]:
model3 = ResNet18()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [82]:
from statistics import mean
import numpy as np

def train(epoch):
    model3.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model3(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'voc07')
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model3.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model3(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target, 'voc07')

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/1083 (0%)]	Loss: 0.924930 	OP: 0.744186	OR: 0.477612	OF1: 0.581818
Train Epoch: 0 [32/1083 (3%)]	Loss: 0.914003 	OP: 0.675000	OR: 0.421875	OF1: 0.519231
Train Epoch: 0 [64/1083 (6%)]	Loss: 0.897528 	OP: 0.560976	OR: 0.333333	OF1: 0.418182
Train Epoch: 0 [96/1083 (9%)]	Loss: 0.885361 	OP: 0.538462	OR: 0.318182	OF1: 0.400000
Train Epoch: 0 [128/1083 (12%)]	Loss: 0.872881 	OP: 0.333333	OR: 0.149254	OF1: 0.206186
Train Epoch: 0 [160/1083 (15%)]	Loss: 0.862220 	OP: 0.354839	OR: 0.171875	OF1: 0.231579
Train Epoch: 0 [192/1083 (18%)]	Loss: 0.847336 	OP: 0.178571	OR: 0.072464	OF1: 0.103093
Train Epoch: 0 [224/1083 (21%)]	Loss: 0.839043 	OP: 0.185185	OR: 0.078125	OF1: 0.109890
Train Epoch: 0 [256/1083 (24%)]	Loss: 0.826515 	OP: 0.120000	OR: 0.044118	OF1: 0.064516


<ipython-input-79-bbf89c8d258d>:40: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [288/1083 (26%)]	Loss: 0.816072 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [320/1083 (29%)]	Loss: 0.808090 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [352/1083 (32%)]	Loss: 0.795985 	OP: 0.185185	OR: 0.074627	OF1: 0.106383
Train Epoch: 0 [384/1083 (35%)]	Loss: 0.786944 	OP: 0.153846	OR: 0.054795	OF1: 0.080808
Train Epoch: 0 [416/1083 (38%)]	Loss: 0.781203 	OP: 0.153846	OR: 0.060606	OF1: 0.086957
Train Epoch: 0 [448/1083 (41%)]	Loss: 0.769401 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [480/1083 (44%)]	Loss: 0.765841 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [512/1083 (47%)]	Loss: 0.756202 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [544/1083 (50%)]	Loss: 0.752000 	OP: 0.041667	OR: 0.014085	OF1: 0.021053
Train Epoch: 0 [576/1083 (53%)]	Loss: 0.747602 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [608/1083 (56%)]	Loss: 0.743030 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [640/1083 (59%)]	Loss: 0.738278 	OP: 0.041667	OR: 0.

<ipython-input-82-59783cd7f140>:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.7064 
OP: 0.000000
OR: 0.000000
OF1: nan

Train Epoch: 1 [0/1083 (0%)]	Loss: 0.705251 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 1 [32/1083 (3%)]	Loss: 0.703731 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 1 [64/1083 (6%)]	Loss: 0.701759 	OP: 0.083333	OR: 0.029851	OF1: 0.043956
Train Epoch: 1 [96/1083 (9%)]	Loss: 0.700672 	OP: 0.041667	OR: 0.016129	OF1: 0.023256
Train Epoch: 1 [128/1083 (12%)]	Loss: 0.701930 	OP: 0.083333	OR: 0.026316	OF1: 0.040000
Train Epoch: 1 [160/1083 (15%)]	Loss: 0.700172 	OP: 0.041667	OR: 0.013699	OF1: 0.020619
Train Epoch: 1 [192/1083 (18%)]	Loss: 0.699418 	OP: 0.083333	OR: 0.027778	OF1: 0.041667
Train Epoch: 1 [224/1083 (21%)]	Loss: 0.700266 	OP: 0.185185	OR: 0.076923	OF1: 0.108696
Train Epoch: 1 [256/1083 (24%)]	Loss: 0.699635 	OP: 0.120000	OR: 0.046154	OF1: 0.066667
Train Epoch: 1 [288/1083 (26%)]	Loss: 0.698465 	OP: 0.185185	OR: 0.076923	OF1: 0.108696
Train Epoch: 1 [320/1083 (29%)]	Loss: 0.696940 	OP: 0.241379	OR: 0

# VGG (frozen)

In [137]:
class MyCNN3(nn.Module):
    def __init__(self):
        super().__init__()
        model_vgg11 = models.vgg11(weights=models.VGG11_Weights.DEFAULT)
        
#         child_counter = 0
#         for child in model_vgg11.children():
#             if child_counter < 1:
#                 for param in child.parameters():
#                     param.requires_grad = False
#                     child_counter += 1
#             else:
#                 child_counter += 1
        
        self.model_pre = nn.Sequential(*list(model_vgg11.children())[:-1])
        fc_feat = model_vgg11.classifier[0].in_features
        self.model_post = nn.Sequential(
            nn.Linear(fc_feat, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
            nn.Linear(4096, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
        )
        self.fc = nn.Linear(4096, len(classes))
    
    def forward(self, X):
        X = self.model_pre(X)
        X = torch.flatten(X,1)
        X = self.model_post(X)
        return self.fc(X)

In [138]:
model4 = MyCNN3()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model4.parameters(), lr=learning_rate)

In [139]:
from statistics import mean
import numpy as np

def train(epoch):
    model4.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model4(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model4.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model4(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

<ipython-input-136-447663f93c68>:28: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [0/947 (0%)]	Loss: 0.696073 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [32/947 (3%)]	Loss: 0.572799 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [64/947 (7%)]	Loss: 0.310805 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [96/947 (10%)]	Loss: 0.440529 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [128/947 (13%)]	Loss: 0.360009 	OP: 0.177778	OR: 0.115942	OF1: 0.140351
Train Epoch: 0 [160/947 (17%)]	Loss: 0.274446 	OP: 0.320000	OR: 0.231884	OF1: 0.268908
Train Epoch: 0 [192/947 (20%)]	Loss: 0.291261 	OP: 0.212766	OR: 0.131579	OF1: 0.162602
Train Epoch: 0 [224/947 (23%)]	Loss: 0.290616 	OP: 0.184211	OR: 0.109375	OF1: 0.137255
Train Epoch: 0 [256/947 (27%)]	Loss: 0.305100 	OP: 0.200000	OR: 0.098592	OF1: 0.132075
Train Epoch: 0 [288/947 (30%)]	Loss: 0.294279 	OP: 0.107143	OR: 0.042857	OF1: 0.061224
Train Epoch: 0 [320/947 (33%)]	Loss: 0.284364 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [352/947 (37%)]	Loss: 0.251522 	OP: 0.000000	OR: 0.000000	OF1

<ipython-input-139-f62952028cc1>:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.2483 
OP: 0.000000
OR: 0.000000
OF1: nan

Train Epoch: 1 [0/947 (0%)]	Loss: 0.234743 	OP: 0.041667	OR: 0.015152	OF1: 0.022222
Train Epoch: 1 [32/947 (3%)]	Loss: 0.250603 	OP: 0.181818	OR: 0.086957	OF1: 0.117647
Train Epoch: 1 [64/947 (7%)]	Loss: 0.234620 	OP: 0.107143	OR: 0.044776	OF1: 0.063158
Train Epoch: 1 [96/947 (10%)]	Loss: 0.248759 	OP: 0.193548	OR: 0.088235	OF1: 0.121212
Train Epoch: 1 [128/947 (13%)]	Loss: 0.227804 	OP: 0.233333	OR: 0.098592	OF1: 0.138614
Train Epoch: 1 [160/947 (17%)]	Loss: 0.250387 	OP: 0.062500	OR: 0.029412	OF1: 0.040000
Train Epoch: 1 [192/947 (20%)]	Loss: 0.219398 	OP: 0.107143	OR: 0.048387	OF1: 0.066667
Train Epoch: 1 [224/947 (23%)]	Loss: 0.226228 	OP: 0.137931	OR: 0.059701	OF1: 0.083333
Train Epoch: 1 [256/947 (27%)]	Loss: 0.269625 	OP: 0.156250	OR: 0.067568	OF1: 0.094340
Train Epoch: 1 [288/947 (30%)]	Loss: 0.243574 	OP: 0.193548	OR: 0.086957	OF1: 0.120000
Train Epoch: 1 [320/947 (33%)]	Loss: 0.219621 	OP: 0.153846	OR: 0

# VGG (Unfrozen)

In [ ]:
class MyCNN4(nn.Module):
    def __init__(self):
        super().__init__()
        model_vgg11 = models.vgg11(weights=models.VGG11_Weights.DEFAULT)
        
        child_counter = 0
        for child in model_vgg11.children():
            if child_counter < 1:
                for param in child.parameters():
                    param.requires_grad = False
                    child_counter += 1
            else:
                child_counter += 1
        
        self.model_pre = nn.Sequential(*list(model_vgg11.children())[:-1])
        fc_feat = model_vgg11.classifier[0].in_features
        self.model_post = nn.Sequential(
            nn.Linear(fc_feat, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
            nn.Linear(4096, 4096, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(.5, inplace = False),
        )
        self.fc = nn.Linear(4096, len(classes))
    
    def forward(self, X):
        X = self.model_pre(X)
        X = torch.flatten(X,1)
        X = self.model_post(X)
        return self.fc(X)

In [ ]:
model5 = MyCNN4()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model5.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model5.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model5(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model5.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model5(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()